In [9]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import glob

In [10]:
def process_taxi_data(input_name):

    taxi_zones=gpd.read_file('NYC_Taxi_Zones.geojson')

    total_regions = len(taxi_zones)
    taxi_zones['passenger_exchange'] = None
    taxi_zones['pu_average_fee'] = None
    taxi_zones['do_average_fee'] = None
    taxi_zones['pu_average_dis'] = None
    taxi_zones['do_average_dis'] = None
    taxi_zones['pu_times'] = None
    taxi_zones['do_times'] = None


    taxi_data=pd.read_csv(input_name)

    location_ids = [str(i) for i in range(0, total_regions + 1)]
    exchange_matrix = pd.DataFrame(index=location_ids, columns=location_ids, data=0.0)
    pu_total_fee=pd.Series(index=location_ids,data=0.0)
    do_total_fee=pd.Series(index=location_ids,data=0.0)
    pu_total_dis=pd.Series(index=location_ids,data=0.0)
    do_total_dis=pd.Series(index=location_ids,data=0.0)
    pu_times=pd.Series(index=location_ids,data=0)
    do_times=pd.Series(index=location_ids,data=0)

    for index,row in taxi_data.iterrows():
        pu_loc=str(row['PULocationID'])
        do_loc=str(row['DOLocationID'])
        
        if int(pu_loc)<=263 and int(do_loc)<=263:
            exchange_matrix.loc[pu_loc,do_loc]-=1*row['passenger_count']
            exchange_matrix.loc[do_loc,pu_loc]+=1*row['passenger_count']

            pu_times.loc[pu_loc]+=1
            do_times.loc[do_loc]+=1

            pu_total_fee.loc[pu_loc]+=row['total_amount']
            do_total_fee.loc[do_loc]+=row['total_amount']
            pu_total_dis.loc[pu_loc]+=row['trip_distance']
            do_total_dis.loc[do_loc]+=row['trip_distance']

    pu_times_cal=pu_times.replace(0,1)
    do_times_cal=do_times.replace(0,1)

    pu_average_fee=pu_total_fee/pu_times_cal
    do_average_fee=do_total_fee/do_times_cal
    pu_average_dis=pu_total_dis/pu_times_cal
    do_average_dis=do_total_dis/do_times_cal

    for index, row in taxi_zones.iterrows():
        loc_id = row['location_id']
        if loc_id in location_ids:
            taxi_zones.at[index,'passenger_exchange']=exchange_matrix.loc[loc_id, :].tolist()
            
            idx=location_ids.index(loc_id)
            taxi_zones.at[index,'pu_average_fee']=pu_average_fee[idx]
            taxi_zones.at[index,'do_average_fee']=do_average_fee[idx]
            taxi_zones.at[index,'pu_average_dis']=pu_average_dis[idx]
            taxi_zones.at[index,'do_average_dis']=do_average_dis[idx]
            taxi_zones.at[index,'pu_times']=int(pu_times[idx])
            taxi_zones.at[index,'do_times']=int(do_times[idx])

    taxi_zones['passenger_exchange']=taxi_zones['passenger_exchange'].apply(lambda x: str(x))

    base_name = os.path.splitext(input_name)[0]  # 这将移除'.csv'

    output_name = base_name + '.geojson'  # 添加新的文件扩展名

    output_dir = 'geojson_split/'
    output_path = os.path.join(output_dir, os.path.basename(output_name))

    os.makedirs(output_dir, exist_ok=True)
    taxi_zones.to_file(output_path,driver='GeoJSON')


In [11]:
def process_all_files(folder_path):
    for file in glob.glob(os.path.join(folder_path, '*.csv')):
        process_taxi_data(file)
        print(f"Processed {file}")

In [12]:
process_all_files('split_data_by_seasons')

C:\Users\ShiRo\AppData\Local\Temp\ipykernel_15948\2456452971.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  taxi_zones.at[index,'pu_average_fee']=pu_average_fee[idx]
C:\Users\ShiRo\AppData\Local\Temp\ipykernel_15948\2456452971.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  taxi_zones.at[index,'do_average_fee']=do_average_fee[idx]
C:\Users\ShiRo\AppData\Local\Temp\ipykernel_15948\2456452971.py:58: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, 

Processed split_data_by_seasons\taxi_autumn_weekdays_00.csv


KeyboardInterrupt: 